In [1]:
"""
Table that will run the skeletons on the full Neurite meshes

1) No longer filters the mesh for only connectedd parts
2) Runs pymeshfix (don't need to do that anymore)
----------------------------------
3) Runs meshlab server script for 
a) Close Holes
b) Select Border
c) Delete Border Vertices
d) Close Holes
e) Remove Duplicate Vertices
f) Remove Zero Area Faces
g) Remove Duplicate Vertices

-- repeat a couple of times --
----------------------------------
4) Run skeletonization

Extra Folders
test_error
pymesh_NEURITES
pymesh_neurons
temp
temp_pymesh
temp_meshlab

"""

"\nTable that will run the skeletons on the full Neurite meshes\n\n1) No longer filters the mesh for only connectedd parts\n2) Runs pymeshfix (don't need to do that anymore)\n----------------------------------\n3) Runs meshlab server script for \na) Close Holes\nb) Select Border\nc) Delete Border Vertices\nd) Close Holes\ne) Remove Duplicate Vertices\nf) Remove Zero Area Faces\ng) Remove Duplicate Vertices\n\n-- repeat a couple of times --\n----------------------------------\n4) Run skeletonization\n\nExtra Folders\ntest_error\npymesh_NEURITES\npymesh_neurons\ntemp\ntemp_pymesh\ntemp_meshlab\n\n"

In [2]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

In [14]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')


In [18]:
pinky.NeuriteRawSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,lagest_mesh_perc the percentage of the entire mesh that the largest submesh makes up
3,648518346341351518,7,=BLOB=,5,0.999109
3,648518346341351523,4,=BLOB=,1,1.0
3,648518346341351525,21,=BLOB=,9,0.998368
3,648518346341351526,7,=BLOB=,4,0.999239
3,648518346341351545,18,=BLOB=,2,0.999757
3,648518346341351635,9,=BLOB=,9,0.997685
3,648518346341351641,9,=BLOB=,4,0.997558
3,648518346341351675,5,=BLOB=,2,0.998364
3,648518346341351695,19,=BLOB=,1,1.0
3,648518346341351710,19,=BLOB=,7,0.999126


In [5]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


""" New read function: for adjusted 2 vert skeleton output"""
def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)



In [6]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/pymesh_NEURITES")) == False:
    os.mkdir("pymesh_NEURITES")

In [7]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_NEURITES"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    print(file_loc)
    print(path_and_filename)
    print(len(vertices))
    print(len(triangles))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [8]:
def meshlab_fix_manifold(key,folder="pymesh_NEURITES"):
    
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [9]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [11]:
@schema
class NeuriteRawSkeleton(dj.Computed):
    definition="""  
    -> pinky.Mesh
    ---
    n_edges   :int unsigned #number of edges stored
    edges     :longblob #array storing edges on each row
    n_bodies    :tinyint unsigned #the amount of segments the neurite was originally split into
    lagest_mesh_perc : float #the percentage of the entire mesh that the largest submesh makes up
    """
    
    
    key_source = pinky.Mesh() & pinky.Neurite() & pinky.CurrentSegmentation

    def make(self, key):
        global_time = time.time()
        #get the mesh with the error segments filtered away
        start_time = time.time()
        print(str(key['segment_id']) +  ":")
        my_dict = (pinky.Mesh & pinky.Neurite.proj() & pinky.CurrentSegmentation
                           & key).fetch1()
        print(f"Step 1: Retrieving Mesh and removing error segments: {time.time() - start_time}")
        new_key = dict(segmentation=key["segmentation"],
                       segment_id=key["segment_id"])
        
        vertices = my_dict["vertices"]
        triangles = my_dict["triangles"]

        #print("Skipping Step 2 with finding mesh pieces and running pymeshfix")
        
        start_time = time.time()
        #pass the vertices and faces to pymeshfix to become watertight
        
        mesh = trimesh_io.Mesh(vertices=my_dict["vertices"], faces=my_dict["triangles"])
        count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
                                                            mesh.edges_sparse,
                                                            directed=False,
                                                            return_labels=True)
        
        new_key["n_bodies"] = count
        values = np.array(labels)
        searchval = 0
        ii = np.where(values == searchval)[0]
        new_key["lagest_mesh_perc"] = len(ii)/len(labels)
        
        print(f"Step 2a: Getting the number of splits: {time.time() - start_time}")
        
              
        start_time = time.time()
        #pass the vertices and faces to pymeshfix to become watertight
        meshfix = pymeshfix.MeshFix(my_dict["vertices"],my_dict["triangles"])
        meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
        print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")
        
        #print("Step 2: Writing Off File")
        vertices = meshfix.v
        triangles = meshfix.f
        
        start_time = time.time()
        #write the new mesh to off file
        path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]),vertices,triangles)
        print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")
        
        #Run the meshlabserver scripts
        start_time = time.time()
        output_mesh = meshlab_fix_manifold(key)
        print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

        print(output_mesh[:-4])
              
        #send to be skeletonized
        start_time = time.time()
        return_value = cm.calcification(output_mesh[:-4])
        if return_value > 0:
            raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                            ' did not finish... exited with error code: ' + str(return_value))
        #print(f"Step 5: Generating Skeleton: {time.time() - start_time}")
        
              
              
        #read in the skeleton files into an array
        bone_array = read_raw_skeleton(output_mesh[:-4]+"_skeleton.cgal")
            
        #print(bone_array)
        if len(bone_array) <= 0:
            raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
        print(f"Step 5: Generating and reading Skeleton: {time.time() - start_time}")
        
              
        start_time = time.time()
        new_key["n_edges"] = len(bone_array)
        new_key["edges"] = bone_array
        #new_key["branches"] = []
        
        #print(key)
        #if all goes well then write to database
        self.insert1(new_key,skip_duplicates=True)
        os.system("rm "+str(path_and_filename)+"*")
        print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
        print(f"Total time: {time.time() - global_time}")
        print("\n\n")
          
                         
                                    

In [12]:
start = time.time()
NeuriteRawSkeleton.populate(reserve_jobs=True)
print(time.time() - start)

648518346341351518:
Step 1: Retrieving Mesh and removing error segments: 0.02868056297302246
Step 2a: Getting the number of splits: 0.02557969093322754
Step 2: Pymesh shrinkwrapping: 0.6879787445068359
/notebooks/Skeleton_Pass_2/pymesh_NEURITES
/notebooks/Skeleton_Pass_2/pymesh_NEURITES/neuron_648518346341351518
20187
40370
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.17165303230285645
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 0.947991132736206
/notebooks/Skeleton_Pass_2/pymesh_NEURITES/neuron_648518346341351518_mls
Step 5: Generating and reading Skeleton: 0.9289419651031494
Step 6: Inserting dictionary: 0.022121191024780273
Total time: 2.8138208389282227



648518346341351523:
Step 1: Retrieving Mesh and removing error segments: 0.025869369506835938
Step 2a: Getting the number of splits: 0.02231764793395996
Step 2: Pymesh shrinkwrapping: 0.8397843837738037
/notebooks/Skeleton_Pass_2/pymesh_NEURITES
/notebooks/Skeleton_Pass_2/pyme

ValueError: Buffer dtype mismatch, expected 'int' but got Python object

In [ ]:
#(schema.jobs & "table_name='__neurite_skeletons'").delete()
#ta3p100.NeuriteSkeletons()#.delete()